In [ ]:
!pip install streamlit
!pip install google-generativeai
!pip install python-dotenv
!pip install langchain
!pip install chromadb
!pip install faiss-cpu
!pip install langchain_google_genai
!pip install langchain-community langchain-core
!pip install pandas

In [41]:
import urllib
import warnings
from pathlib import Path as p
import pandas as pd
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')
api_key_=userdata.get('api_key')
warnings.filterwarnings("ignore")
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model="gemini-2.0-flash",google_api_key=api_key_,temperature=0.3,convert_system_message_to_human=True)

def get_text_chunks(text):
  text_splitter=RecursiveCharacterTextSplitter(chunk_size=10000,chunk_overlap=1000)
  chunks=text_splitter.split_text(text)
  return chunks
df=pd.read_csv("/content/Maude_Export.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
fa=[]
for i in range(5):
  s=df.iloc[[i],[0]].to_string()
  chunks=get_text_chunks(s)
  embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=api_key_)
  vector_index=Chroma.from_texts(chunks,embeddings).as_retriever(search_kwargs={"k":10})
  qa_chain=RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True
  )
  question="which is the product failure in this complaint and provide only failure name in the answer"
  result=qa_chain({"query":question})
  fa.append(result["result"])
print(fa)


['FAILED PREVENTIVE MAINTENANCE', 'FAILS VOLUME AND PATIENT SIDE OCCLUSION', 'FAILS VOLUME AND PATIENT SIDE OCCLUSION', 'FAILS VOLUME AND PATIENT SIDE OCCLUSION', 'FAILS VOLUME AND PATIENT SIDE OCCLUSION']
